# CSE 547 HW 1
## Michael Berlinger

In [0]:
# Please enter userlist below
userlist=[924, 8941, 8942, 9019, 9020, 9021, 9022, 9990, 9992, 9993]

### Set-up

In [0]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 217.8MB 62kB/s 
     |████████████████████████████████| 204kB 31.7MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.5-py2.py3-none-any.whl size=218257927 sha256=bc2b5c0e5ba57b436f2179be44a1d8790cc7c7113f2da287aedf70b4036a5d6c
  Stored in directory: /root/.cache/pip/wheels/bf/db/04/61d66a5939364e756eb1c1be4ec5bdce6e04047fc7929a3c3c
Successfully built pyspark
openjdk-8-jdk-headless is already the newest version (8u242-b08-0ubuntu3~18.04).
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.


In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
id='1jc3CqVlNXWrfRH37drCoE3lasbvtYPC4'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('soc-LiveJournal1Adj.txt')

### Libraries

### Set-up Spark

In [0]:
# create the Spark Session
conf=SparkConf().set("spark.executor.memory", "8g").set("spark.ui.port", "4050")

# create the Spark Context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()


In [0]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

--2020-04-13 19:05:48--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.207.7.190, 52.73.120.139, 52.5.232.178, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.207.7.190|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.13M  19.2MB/s    in 0.7s    

2020-04-13 19:05:49 (19.2 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
https://c399558b.ngrok.io


### Code Body

In [0]:
data = sc.textFile('soc-LiveJournal1Adj.txt')

In [0]:
#Map the data to: <user>,<[f1,f2,...,fn]>

users = data\
  .map(lambda x: x.split('\t'))\
  .map(lambda y: (y[0], y[1].split(',')))

#Move to df
users=spark.createDataFrame(users)

#Explode friend array to rows: <user>,<friend>
users=users\
  .withColumnRenamed('_1','user')\
  .withColumn('friend',explode('_2'))\
  .drop('_2')

if not userlist:
  sel_users=users
else: sel_users=users.where(col('user').isin(userlist))

In [0]:
users=users.select(
    users.user.cast('integer'),
    users.friend.cast('integer')
)

In [0]:
#Join along matching friends, removing users being recommended to themselves
users2=sel_users.join(
  users.alias('a').withColumnRenamed('user','recommendation'),
  'friend',
  'left'
).where(col('user')!=col('recommendation'))

#Remove instances where recommendations are already friends
users3=users2.join(users.alias('a').withColumnRenamed('friend','recommendation'),
  ['user','recommendation'],
  'left_anti'
)

#Count Mutual Friends
users4=users3.groupby(['user','recommendation'])\
  .agg(count("*").alias('mutual_count'))

In [0]:
users5=users4\
  .withColumn('rank',row_number()\
    .over(
        Window.orderBy(col('mutual_count').desc(),col('recommendation'))\
        .partitionBy('user')
    )
  ).filter(col('rank')<=10)

In [0]:
user6=users5\
  .drop('mutual_count')\
  .withColumn('list',collect_list('recommendation')\
    .over(
      Window.partitionBy('user').orderBy('rank')
    )
  ).groupBy('user').agg(max('list').alias('recommendations'))

In [0]:
out=user6.rdd.map(
  lambda a: (
    a['user'] + '\t' + ','.join(str(r) for r in a['recommendations'] )
  )
)

### Output

In [0]:
for line in out.collect():
  print(line)


9993	9991,13134,13478,13877,34299,34485,34642,37941
9019	9022,317,9023
924	439,2409,6995,11860,15416,43748,45881
8942	8939,8940,8943,8944
9020	9021,9016,9017,9022,317,9023
9021	9020,9016,9017,9022,317,9023
8941	8943,8944,8940
9992	9987,9989,35667,9991
9990	13134,13478,13877,34299,34485,34642,37941
9022	9019,9020,9021,317,9016,9017,9023
